### Portail meteo.data - Téléchargement & affichage des dernières données MENSUELLES de Météo-France (Métropole & outre-mer)
Une connexion internet est nécessaire pour accéder aux archives des données à l'url ci-dessous

Les "dernières" données (latest) correspondent aux fichiers mis à jour chaque jour, et qui vont du mois de janvier de l'année précédente au mois en cours même partiel (pour les données mensuelles)

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)<br>
Fichier CSV descriptif champs: https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e


Utilisez mon autre script pour visualiser sous forme de carte la liste des postes météorologiques fournie par Météo-France sous forme de fichier JSON https://meteo.data.gouv.fr/https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922

https://github.com/loicduffar

In [163]:
# Télécharge les les archives ZIP & et décompresse les fichiers CSV (DENIERE PERIODE DISPONIBLE)
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Personalisation ====================
# Chemin d'enregistrement des archives gz et des fichiers CSV décompressés
folder_gz= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M"
folder_csv= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M"

# Numéro des Postes météo souhaités (chaine de 8 caractères) - voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922
# LES DEPARTEMENTS CORRESPONDANTS DOIVENT ËTRE PRESENTS DANS LA LISTE DES URLS (voir ======= Initialisation ==========)
postes= ['04088001', '04039001', '04230001',                  # Forcalquier, Castellane, Valensole \
        '05046001',                                           # Embrun \
        # '06088001', '06029001',                               # Nice, Cannes \
        '13001009', '13111002', '13103001','13055001',        # Aix en Provence, Vauvenargue, Salon, Marseille, \
        '83031001', '83061001', '83137001',                   # Le Luc, Fréjus, Toulon \
        '84003002', '84009002']                               # Apt-Viton, Bastide des Jourdans

# ================ Extrait du fichier data CSV =============
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMM	RR	QRR	NBRR	RR_ME	RRAB	QRRAB	RRABDAT	NBJRR1	NBJRR5	NBJRR10	NBJRR30
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202201	3.6	1	31		1.4	1	4	1	0	0	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202202	26.1	1	28		24.7	1	14	2	1	1	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202203	6.6	1	31		3.6	1	30	2	0	0	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202204	27	1	30		17.6	1	23	4	2	1	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202205	26	1	31		16.8	1	8	3	2	1	0

# ================ Extrait du fichier data CSV Descriptif des champs les plus souvent utiles (précipitations et températures) =============
# NUM_POSTE       	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL       	 nom usuel du poste
# LAT             	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON             	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI            	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMM          	 mois
# RR              	 cumul mensuel des hauteurs de précipitation (en mm et 1/10)
# QRR             	 code qualité de RR
# NBRR            	 nombre de valeurs présentes de hauteur de précipitation quotidienne
# RR_ME           	 cumul mensuel estimé des hauteurs de précipitation (en mm et 1/10)
# RRAB            	 précipitation maximale tombée en 24 heures au cours du mois
# QRRAB           	 code qualite de RRAB
# TX              	 moyenne mensuelle des températures maximales (TX) quotidiennes (en °C et 1/10)
# QTX             	 code qualité de TX
# TN              	 moyenne mensuelle des températures minimales (TN) quotidiennes (en °C et 1/10)
# QTN             	 code qualité de TN
# TM              	 moyenne mensuelle des (TN+TX)/2 quotidiennes (en °C et 1/10)
# QTM             	 code qualité de TM
# TMM             	 moyenne mensuelle des températures moyennes (TM) quotidiennes (en °C et 1/10)
# QTMM            	 code qualité du TMM
# TMMIN           	 minimum mensuel des moyennes (TN+TX)/2 quotidiennes (en °C et 1/10)
# QTMMIN          	 code qualité de TMMIN
# TMMAX           	 maximum mensuel des moyennes (TN+TX)/2 quotidiennes (en °C et 1/10) 
# QTMMAX          	 code qualité du TMMAX

# définit un dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
# Lit en ligne le fichier "MENSQ_descriptif_champs.csv" de description des champs et le stocke dans un dataframe pandas
url = "https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e"
df_desc = pd.read_csv(url, sep=":", header= None, index_col=0, names= ["param", "name_long"], dtype={"param":str, "name_long":str}, encoding= 'utf-8')
# display(df_desc)

# ================ Initialisation ====================
# Structure du nom des fichiers de données mensuelles (1 fichier par département dont le numéro sera ajouté automatiquement au début et à la fin du template ci-dessous)
template_end= '_latest-2022-2023.csv'
template_start= 'MENSQ_'

# ---------------- urls de téléchargement des archives des dernières données depuis janvier de l'année précédente https://meteo.data.gouv.fr/ (Ajouter d'autres départements si besoin)
urls= dict()
urls['04']= "https://www.data.gouv.fr/fr/datasets/r/5c2d6f73-4162-4f36-bb41-98d7bc3e2ee1" 
urls['05']= 'https://www.data.gouv.fr/fr/datasets/r/589b53fd-adef-44a4-b306-1dee1500d2d2'
urls['06']= 'https://www.data.gouv.fr/fr/datasets/r/f8bdc686-8c37-4ced-8c36-407148bca194'
urls['13']= 'https://www.data.gouv.fr/fr/datasets/r/78fe3a8a-7f32-44dd-a1b5-8acb841d6cc1'
urls['83']= 'https://www.data.gouv.fr/fr/datasets/r/dc5ee912-e84a-4f15-bd08-7f326795ce41'
urls['84']= 'https://www.data.gouv.fr/fr/datasets/r/75526d6c-81d5-4aad-8961-e9fc2a43f7b5'
# print(urls)

# ---------------- Définition des fonctions
def convert_to_date(aaaamm):
    return pd.to_datetime(str(aaaamm), format='%Y%m', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
        # print("Téléchargement terminé")
    else:
        print("Fichier non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier non trouvé: ", file)

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    print('Lecture: ', file)
    df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMM':str}, parse_dates=['AAAAMM'], date_parser= convert_to_date)
    return df

# ================ Traitement ====================
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"))
# fait la liste des départements concernés et garde les 2 premiers chiffres uniques
# On récupère les 2 premiers chiffres de chaque poste
departements = []
for poste in postes:
    departements.append(poste[:2])
# On ne garde que départements uniques
departements = list(set(departements)) # set() ignore automatiquement les doublons (ce qui évite de passer par array numpy pour utiliser la fonction np.unique() )
# On trie la liste
departements.sort()
# On affiche la liste
print('Départements concernés: ', departements)

# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
for i, departement in enumerate(departements):
    # On récupère l'url
    url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
    filename = f"{template_start}{departement}{template_end}"
    download_file(url, filename)
    decompress_gz(filename)
    # Lecture du fichier CSV dans un dataframe pandas
    # filename = os.path.join(folder_csv, f"{template_start}{departement}{template_end}")

    df_departement= read_csv(filename)
    # df_departement = pd.read_csv(filename, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMM':str}, parse_dates=['AAAAMM'], date_parser= convert_to_date)
    df_departement['NUM_POSTE'] = df_departement['NUM_POSTE'].str.strip()
    if i== 0:
        # initialisation du dataframe final df avec les données du premier département        
        df= df_departement
    else:
        df= pd.concat([df, df_departement])        

display(df)

2023-12-19 20:16
Départements concernés:  ['04', '05', '13', '83', '84']
Téléchargement:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M\MENSQ_04_latest-2022-2023.csv.gz
Décompression X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_04_latest-2022-2023.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_04_latest-2022-2023.csv
Téléchargement:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M\MENSQ_05_latest-2022-2023.csv.gz
Décompression X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_05_latest-2022-2023.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,RR,QRR,NBRR,RR_ME,...,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBJGREL,NBJORAG,NBJBROU
0,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-01,4.2,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-02-01,29.4,1.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-03-01,13.0,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-04-01,53.6,1.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-05-01,38.0,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,84150001,VISAN,44.336667,4.905500,141,2023-08-01,49.0,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,84150001,VISAN,44.336667,4.905500,141,2023-09-01,56.9,1.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,84150001,VISAN,44.336667,4.905500,141,2023-10-01,131.1,1.0,31.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,84150001,VISAN,44.336667,4.905500,141,2023-11-01,44.6,1.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Tracé du graphique des précipitations

In [164]:
# trace un graphique plotly de df avec la colonne 'AAAAMM' en abscisse et la colonne 'RR' en ordonnée
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

#-------------- initialisation des paramètres du graphique
df_postes= df[df['NUM_POSTE'].isin(postes)]

color_sequence_bar = plotly.colors.qualitative.Set3
# définit une séquence de couleurs inversée pour les courbes cumulées
# color_sequence_scatter = color_sequence_bar[::-1]
color_sequence_scatter = plotly.colors.qualitative.Dark24

specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1, shared_xaxes= True,
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['AAAAMM'], y= df_poste['RR'], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)]),  # Utilisez la couleur correspondante de la séquence
                         ),
                row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df_poste['AAAAMM'], y= df_poste['RR'].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color=color_sequence_scatter[i % len(color_sequence_scatter)]),  # Utilisez la couleur correspondante de la séquence
                             ), 
                 row=i+1, col=1, secondary_y= True)

fig.update_layout(title_text= 'meteo.data - ' + "Précipitations mensuelles (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, height= 133.*len(postes), width= 1000,
                    hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                    )
# Trace une ligne verticale au travers tous les subplots matérialisant l'abscisse survolée par la souris 
xlast= 'x' + str(len(postes))
fig.update_traces(xaxis= xlast)
max_value= df_postes['RR'].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()['RR'].max()
fig.update_yaxes(range= [0, max_value], secondary_y= False)  # Replace max_value with the desired maximum coordinate
fig.update_yaxes(title_text= "Cumulé", 
                 range= [0, max_value_cum], secondary_y= True)
# sauvegarde le graphique dans un fichier HTML
fig.write_html(os.path.join(folder_csv, "meteo.data MENS" + template_end[:-4] + ".html"))
fig.show()

In [161]:
postes

['05046001',
 '06088001',
 '06029001',
 '13001009',
 '13111002',
 '13103001',
 '13055001',
 '83031001',
 '83061001',
 '83137001',
 '84003002',
 '84009002']

In [133]:
[ df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0] for poste in postes ]

IndexError: single positional indexer is out-of-bounds